In [ ]:
import cv2
import os

def draw_bbox_on_image(image_path, txt_path):
    """
    Draw YOLO bounding boxes from a .txt file on an image.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Read bounding box data from the txt file
    with open(txt_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            _, x_center, y_center, width, height = map(float, line.strip().split())

            # Denormalize the coordinates
            x_center *= image.shape[1]
            y_center *= image.shape[0]
            width *= image.shape[1]
            height *= image.shape[0]

            # Convert to top-left x, y coordinates
            x_min = int(x_center - width / 2)
            y_min = int(y_center - height / 2)
            x_max = int(x_center + width / 2)
            y_max = int(y_center + height / 2)

            # Draw the bounding box on the image
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green color for the bounding box

    # Display the image
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
import os
def remove_superpixels(folder):
    """
    Remove all superpixel files from a folder.
    """
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    for file in files:
        if file.endswith('superpixels.png'):
            os.remove(os.path.join(folder, file))

In [ ]:
import os
import cv2
import numpy as np

def compute_yolo_bbox(binary_mask):
    """
    Compute the YOLO bounding box for a binary mask.
    """
    # Find the coordinates of non-zero pixels
    rows, cols = np.where(binary_mask == 255)

    # Get the minimum and maximum coordinates
    x_min, x_max = np.min(cols), np.max(cols)
    y_min, y_max = np.min(rows), np.max(rows)

    # Compute the center, width, and height
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min

    return x_center, y_center, width, height


In [ ]:
import os
import cv2
import pandas as pd

def create_yolo_labels(seg_folder_path, cls_folder_path, dst_folder_path):
    """
    Iterate through the binary images in the folder, generate YOLO bounding boxes, 
    and determine the class from the dataframe.
    """
    labels = pd.read_csv(cls_folder_path, index_col=0)

    for filename in os.listdir(seg_folder_path):
        if filename.endswith(".png"):
            # Extract image_id from filename
            image_id = filename[:12]

            # Determine the class based on the dataframe
            class_row = labels.loc[image_id]
            if class_row['melanoma'] == 1.0:
                class_id = 0
            elif class_row['seborrheic_keratosis'] == 1.0:
                class_id = 1
            else:
                class_id = 2  # Assuming a third class for 'other' or 'nevi' based on the given example

            # Load the binary image
            image_path = os.path.join(seg_folder_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Compute the YOLO bounding box
            x_center, y_center, width, height = compute_yolo_bbox(image)
            
            # Normalize the coordinates
            x_center /= image.shape[1]
            y_center /= image.shape[0]
            width /= image.shape[1]
            height /= image.shape[0]

            # Write the bounding box to a .txt file
            output_path = os.path.join(dst_folder_path, filename.replace(".png", ".txt"))
            with open(output_path, "w") as file:
                file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")



In [ ]:
cls_folder_path = "/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/ISIC_labels.csv"
seg_folder_path = "/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/segmentation"
dst_folder_path = "/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/yolo_labels"

create_yolo_labels(seg_folder_path, cls_folder_path, dst_folder_path)

In [ ]:
import os
import shutil

def move_contents_to_folder(source_folder, destination_folder):
    """
    Move the contents of the source folder into the destination folder.
    
    Parameters:
    - source_folder: Path to the source folder whose contents will be moved.
    - destination_folder: Path to the destination folder where the content will be moved to.
    """
    
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)
    
    # Move content from source_folder to destination_folder
    for item in os.listdir(source_folder):
        source_item = os.path.join(source_folder, item)
        destination_item = os.path.join(destination_folder, item)
        
        # Move the item to the destination
        shutil.move(source_item, destination_item)


In [ ]:
move_contents_to_folder("/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/val", "/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/train")

In [6]:
img_path = "/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/imgs/ISIC_0000000.jpg"

#Open the image and check the size
import cv2
img = cv2.imread(img_path)

print(img.shape)

import os

def get_image_size(image_path):
    """
    Get the size of an image.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Return the image size
    return image.shape[0], image.shape[1], img.shape[2]

for file in os.listdir("/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/imgs"):
    print(get_image_size(os.path.join("/Users/larsmoan/Documents/UQ/COMP3710/PatternAnalysis-2023/lesion_detection_larsmoan/data/ISIC_2017/imgs", file)))

(767, 1022, 3)
(2592, 3872, 3)
(1129, 1504, 3)
(768, 1024, 3)
(1129, 1504, 3)
(768, 1024, 3)
(2000, 3008, 3)
(2848, 4288, 3)
(4402, 6602, 3)
(2016, 3024, 3)
(2848, 4288, 3)
(2592, 3872, 3)
(1129, 1504, 3)
(768, 1024, 3)
(4419, 6668, 3)
(2592, 3872, 3)
(2848, 4288, 3)
(768, 1024, 3)
(2016, 3024, 3)
(2848, 4288, 3)
(768, 1024, 3)
(768, 1024, 3)
(2000, 3008, 3)


KeyboardInterrupt: 